In [84]:
import os
import pandas as pd
import datetime
import numpy as np


In [101]:
list_of_files = files=os.listdir("RAW")
for _ in list_of_files:
    Company = pd.read_csv('RAW/'+_)
    bse_sensex = pd.read_csv("BSE-Sensex.csv")
    nse = pd.read_csv("Nse.csv")

    Company['Date'] = pd.to_datetime(Company['Date'])
    bse_sensex['Date'] = pd.to_datetime(bse_sensex['Date'])
    nse['Date'] = pd.to_datetime(nse['Date'])
    
    # Retrive Specific Range

    mask_Company = (Company['Date']>'2008-01-01')&(Company['Date']<'2016-12-31')
    Company = Company.loc[mask_Company]
    mask_bse_sensex = (bse_sensex['Date']>'2008-01-01')&(bse_sensex['Date']<'2016-12-31')
    bse_sensex = bse_sensex.loc[mask_bse_sensex]
    mask_nse = (nse['Date']>'2008-01-01')&(nse['Date']<'2016-12-31')
    nse = nse.loc[mask_nse]

    
    date = list(Company['Date'])
    Company_close = list(Company['Adj Close'])
    Company_volume=list(Company['Volume'])
    bse_sensex_close = list(bse_sensex['Adj Close'])
    bse_sensex_volume=list(bse_sensex['Volume'])
    nse_close = list(nse['Adj Close'])
    
    
    #Create Full Set of dates

    Full_Dates = makedate(datetime.datetime(2008,1,1),datetime.datetime(2016,12,31),datetime.timedelta(seconds = 3600*24))

    
    
    # Add missing Dates
    Company_close_new = []
    Company_volume_new = []
    bse_sensex_close_new = []
    bse_sensex_volume_new = []
    nse_close_new = []
    i=0
    j=0
    while i<len(Full_Dates):
        if j<len(date) and date[j]==Full_Dates[i]:
            Company_close_new.append(Company_close[j])
            Company_volume_new.append(Company_volume[j])
            bse_sensex_close_new.append(bse_sensex_close[j])
            bse_sensex_volume_new.append(bse_sensex_volume[j])
            nse_close_new.append(nse_close[j])
            j+=1
        else:
            if j==0:
                Company_close_new.append(Company_close[j])
                Company_volume_new.append(Company_volume[j])
                bse_sensex_close_new.append(bse_sensex_close[j])
                bse_sensex_volume_new.append(bse_sensex_volume[j])
                nse_close_new.append(nse_close[j])
            else:
                Company_close_new.append(Company_close[j-1])
                Company_volume_new.append(Company_volume[j-1])
                bse_sensex_close_new.append(bse_sensex_close[j-1])
                bse_sensex_volume_new.append(bse_sensex_volume[j-1])
                nse_close_new.append(nse_close[j-1])

        i+=1
        
    #Normalization in range 0-10000
    
    SMA_list = SMA(preprocess(Company_close_new,0))
    EMA_list = EMA(preprocess(Company_close_new,0),15)
    MACD_list = MACD(preprocess(Company_close_new,0))
    RSI_List = RSI(preprocess(Company_close_new,0))
    AROS_List = AROS(preprocess(Company_close_new,0))
    
    

    Company_close_new = normalize(Company_close_new)[40:]
    Company_volume_new = normalize(Company_volume_new)[40:]
    bse_sensex_close_new = normalize(bse_sensex_close_new)[40:]
    bse_sensex_volume_new = normalize(bse_sensex_volume_new)[40:]
    nse_close_new = normalize(nse_close_new)[40:]
    SMA_list = normalize(SMA_list)[40:]
    EMA_list = normalize(EMA_list)[40:]
    MACD_list = normalize(MACD_list)[40:]
    RSI_List = normalize(RSI_List)[40:]
    AROS_List = normalize(AROS_List)[40:]
    

    Final_Frame = pd.DataFrame(Full_Dates[40:],columns=["Date"])
    Final_Frame[_+"-CLOSE"] = Company_close_new
    Final_Frame[_+"-VOLUME"] = Company_volume_new
    Final_Frame["BSE-CLOSE"] = bse_sensex_close_new
    Final_Frame["BSE-VOLUME"] = bse_sensex_volume_new
    Final_Frame["NSE-CLOSE"] = nse_close_new
    Final_Frame["SMA"] = SMA_list
    Final_Frame["EMA"] = EMA_list
    Final_Frame["MACD"] = MACD_list
    Final_Frame["RSI"] = RSI_List
    Final_Frame["AROS"] = AROS_List
    
    del Final_Frame['Date']

    

    target_signal_binary=[]

    i=1
    while i<len(Final_Frame):
        if Final_Frame[_+"-CLOSE"][i]>Final_Frame[_+"-CLOSE"][i-1]:
            target_signal_binary.append(np.array([1,0]))
        else:
            target_signal_binary.append(np.array([0,1]))
        i+=1

    target_signal_binary = np.array(target_signal_binary[29:])  


    #Feed = []

    #i=0
    #while i<len(Final_Frame)-30:
    #    Feed.append(np.array(Final_Frame[i:i+30]))
    #    i+=1
    #Feed = np.array(Feed)
    Feed = np.array(Final_Frame)
    
    shuffle2geather(Feed,target_signal_binary)
    pickle_data(Feed, target_signal_binary, 'Processed/randomforest-'+_[:-4])
    


In [105]:
Feed

array([[ 867,  244, 8553, ..., 1197, 1643,  833],
       [ 441,  235, 7610, ..., 1217, 5117, 2916],
       [2735,  151,  843, ..., 1184, 2093,    0],
       ..., 
       [1794,   36, 4511, ..., 1212, 3877, 6250],
       [ 995,  564, 9686, ..., 1215, 6286, 3750],
       [ 841,  147, 3815, ..., 1190,    0,  833]])

In [104]:
target_signal_binary

array([[0, 1],
       [0, 1],
       [1, 0],
       ..., 
       [0, 1],
       [1, 0],
       [0, 1]])

In [103]:
Final_Frame

,MTNL.BO.csv-CLOSE,MTNL.BO.csv-VOLUME,BSE-CLOSE,BSE-VOLUME,NSE-CLOSE,SMA,EMA,MACD,RSI,AROS
0,5637,506,4323,848,4011,6330,6575,1388,3538,2083
1,5523,530,3935,885,3604,6291,6520,1378,3374,2708
2,5293,410,3925,627,3575,6238,6445,1360,2963,208
3,5438,356,4083,618,3716,6196,6396,1355,4475,208
4,5705,472,4463,627,4137,6201,6384,1366,5949,416
5,5797,428,4625,765,4293,6225,6384,1376,5713,1666
6,5797,428,4625,765,4293,6237,6385,1381,5713,1666
7,5797,428,4625,765,4293,6249,6385,1382,5713,1666
8,5804,255,4594,507,4253,6262,6387,1381,4713,1666
9,5682,715,4607,553,4259,6247,6373,1371,3981,1666


In [85]:
def makedate(start, end, step):
    import datetime
    import pandas as pd
    Full_Dates = []

    while start < end:
        Full_Dates.append(start.strftime('%Y-%m-%d %H:%M:%S'))
        start += step

    return pd.to_datetime(Full_Dates)

In [86]:
def preprocess(ls,const):
    out=[]
    i=0
    while i<len(ls):
        try:
            ls[i] = float(ls[i])
            out.append(ls[i])
        except ValueError:
            out.append(out[i-1])
        i+=1
    return out

In [87]:
def normalize(ls):
    ls = preprocess(ls,0)
    ls = np.array(ls)
    ls = (ls-np.amin(ls))*10000/(np.amax(ls)-np.amin(ls))
    return np.array(ls,'int')

In [88]:
def shuffle2geather(A,B):
    import numpy as np
    RNG_state = np.random.get_state()
    np.random.shuffle(A)
    np.random.set_state(RNG_state)
    np.random.shuffle(B)

In [89]:
def pickle_data(X, Y, Name):
    import pickle
    final_list = {'X':X, 'Y':Y}
    with open(Name,'wb') as f:
        pickle.dump(final_list,f)

In [90]:
def SMA(ls):
    import numpy as np
    out=[0]*14
    ls = np.array(ls)
    i=0
    while i+15<=len(ls):
        out.append(np.average(ls[i:i+15]))
        i+=1
    return np.array(out)

In [91]:
def EMA(ls, period):
    import numpy as np
    out=[0]*period
    ls = np.array(ls)
    i=0
    multiplier = 2/(1+period+1)
    while i+period+1<=len(ls):
        if i==0:
            out.append(np.average(ls[i:i+period+1]))
        else:
            out.append(((ls[i+period]-out[i+period-1])*multiplier)+out[i+period-1])
        i+=1
    return np.array(out)

In [92]:
def MACD(ls):
    macd_line = EMA(ls,12) - EMA(ls,26)
    ema_9 = EMA(macd_line[26:],9)
    signal = np.append(np.array([0]*26),ema_9)
    return np.array(macd_line-signal)
    

In [93]:
def RSI(ls):
    import numpy as np
    change=[0]
    Gain=[0]
    Loss=[0]
    i=1
    while i<len(ls):
        temp = ls[i]-ls[i-1]
        change.append(temp)
        if temp>0:
            Gain.append(temp)
            Loss.append(0)
        elif temp<0:
            Gain.append(0)
            Loss.append(-temp)
        else:
            Gain.append(0)
            Loss.append(0)
        i+=1
    Gain = np.array(Gain)
    Loss = np.array(Loss)
    
    RSI = [0]*13
    
    
    i=0
    while i+14<=len(ls):
        if np.average(Loss[i:i+14])==0:
            RSI.append(100)
        else:
            RSI.append((100-(100/(1+(np.average(Gain[i:i+14])/np.average(Loss[i:i+14]))))))
        i+=1
    return np.array(RSI)

In [94]:
def AROS(ls):
    import numpy as np  
    up = [0]*24
    down = [0]*24
    i=24
    while i<len(ls):
        temp = np.array(ls[i-24:i+1])
        temp = np.flip(temp,0)
        up.append(np.argmax(temp))
        down.append(np.argmin(temp))
        i+=1
    return (np.array(down)-np.array(up))*4

In [77]:
import numpy as np
ls = np.array([10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,50,60,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,50,60,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,50,60])
ls2 = np.array([20,10,10,10,-10,10,10,10,10,10,10,10,10,10,10,-50,10])
avg = np.average(ls)